In [1]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from rna_seq_normalization import Normalization as Norm

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
specie = "human"
GENOMEDIR = "/genomics/users/marta/genomes"

### cancer data
cancer_dir = users_dir + "/cancers"
raw_cancer_dir="/users/genomics/marta/TCGA_RNASeq"
tcga_projects=["TCGA-BRCA","TCGA-LUSC","TCGA-PRAD","TCGA-KIRC","TCGA-KIRP","TCGA-LUAD"]#,"TCGA-BLCA","TCGA-LIHC"]
other_projects=["GSE102101_KIRC","GSE133624_BLCA","GSE214846_LIHC","GSE22260_PRAD","GSE229705_LUAD","GSE89223_PRAD","PRJEB2449_PRAD","SRP107326_COAD","SRP238334_KIRC","TCGA_COAD","GSE103001_BRCA"]
all_projects = tcga_projects + other_projects

## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/1transcript_1gene.reconstructed.csv")

def count_greater_than_one(row):
    return (row > 1).sum()

## Quantify with TCGAData and new reference
`featureCounts_newRef_cancer.sh`

output in: `/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/cancers/featureCounts`

## TPM

In [9]:
for file in os.listdir(os.path.join(cancer_dir,"featureCounts")):
    if file.endswith(".txt"):

        proj = file.split("featureCounts_")[-1]
        cancer_type = proj[:-4]
        print(cancer_type)
        toc = pd.read_csv(os.path.join(cancer_dir,"featureCounts",file), sep="\t", comment="#")
        toc['Geneid']=toc['Geneid'].str.split('.').str[0]
        toc.rename(columns={'Geneid':'transcript_id'}, inplace=True)

        filter_col = [col for col in toc if col.startswith('/')]
        for col in filter_col:
            new_col=col.split("Aligned")[0]
            new_col=new_col.split("/")[-1]
            if proj in tcga_projects:
                new_col=cancer_type+new_col[4:]
            toc.rename(columns={col:new_col}, inplace=True)

        length = toc['Length']
        genes = toc['transcript_id']
        # we are only interested in the columns with counts
        counts = toc
        counts.drop(["Chr","Start","End","Strand","Length","transcript_id"],axis=1, inplace=True)
        # calculate TPMs
        tpm_df = Norm.tpm(counts, length)
        # add transcript_id and length again
        tpms = pd.concat([genes,tpm_df, length], axis=1)
        tpms.to_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+cancer_type+".csv", index=None)
 

TCGA-PRAD
TCGA-LUSC
GSE102101_KIRC
BLCA
TCGA-BRCA
TCGA-KIRP
GSE22260_PRAD
GSE133624_BLCA
PRJEB2449_PRAD
TCGA-LUAD
LIHC
TCGA-KIRC
SRP238334_KIRC


In [10]:
## filter patients we are interested in, from those not TCGA
for proj in all_projects:
    if proj in tcga_projects:
        cancer_type = proj[5:]
        print(cancer_type)

        fc = pd.read_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+proj+".csv")
        tumor_transcripts = list()
        patients=pd.read_csv(os.path.join("/users/genomics/marta/TCGA_RNASeq",proj,"results/QC_patients.csv"))
        patients['normal'] = patients['normal'].str.replace('TCGA', cancer_type)
        patients['tumor'] = patients['tumor'].str.replace('TCGA', cancer_type)

    elif proj in other_projects:
        fc = pd.read_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+proj+".csv")
        print(fc)
        tumor_transcripts = list()
        patients=pd.read_csv(os.path.join("/users/genomics/marta/cancers_RNASeq",proj,"results/patients.csv"))

        samples_to_keep = patients.tumor.values.tolist() + patients.normal.values.tolist() + ['Length','trnascript_id']
        fc_selected = fc[fc.columns.intersection(samples_to_keep)]


    for index,patient in patients.iterrows():
        if proj in tcga_projects:
            if "_" not in patient.iloc[0]: ## if there are two samples per patient, keep only one
                patient_fc = fc[["transcript_id","Length",patient.iloc[2]]]

                tumor_patient_fc = patient_fc[patient_fc[patient.iloc[2]] >= 1 ]
                tumor_transcripts.extend(tumor_patient_fc.transcript_id.values.tolist())
        if proj in other_projects:
            patient_fc = fc[["transcript_id","Length",patient.iloc[2]]]

            tumor_patient_fc = patient_fc[patient_fc[patient.iloc[2]] >= 1 ]
            tumor_transcripts.extend(tumor_patient_fc.transcript_id.values.tolist())            

    tumor1FPKM = fc[fc['transcript_id'].isin(tumor_transcripts)]

    known = tumor1FPKM[tumor1FPKM['transcript_id'].str.contains('ENST')]
    novel = tumor1FPKM[tumor1FPKM['transcript_id'].str.contains('TCONS')]
    merged = tumor1FPKM.merge(transcript_gene, on=['transcript_id'], how="inner")
    merged = merged[merged['gene_type'].isin(['lncRNA','processed_pseudogene','novel','protein_coding'])]

    lncRNA = merged[merged['gene_type'].isin(['lncRNA','processed_pseudogene'])]
    print("lncRNA: ",len(lncRNA))

    cds = merged[merged['gene_type'] == "protein_coding"]
    print("PROTEIN CODING: ",len(cds))

    novel = merged[merged['gene_type'] == "novel"]
    novel = novel[novel['Length'].astype(int) < 91667]
    novel = novel[novel['Length'].astype(int) > 300]

    print("NOVEL: ",len(novel))

    # # tumor1FPKM = pd.concat([novel, lncRNA, cds], ignore_index = True)
    # merged.to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_table_of_counts_")+cancer_type+".csv",index=False)
    # merged[['transcript_id','gene_id','gene_name','gene_type','Length']].to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_")+cancer_type+".csv",index=False)

    tumorONLY_merged = merged[merged.columns.drop(list(merged.filter(regex='normal')))]
    ###### GET TUMOR-EXPRESSED > 1 TPM
    # Filter only integer columns
    int_columns = tumorONLY_merged.select_dtypes(include='int64')

    # Create a new column with the count of int columns per row whose value is > 1
    # tumorONLY_merged['n'] = int_columns.apply(lambda row: (row > 1).sum(), axis=1)
    tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)
    print(tumorONLY_merged)
    tumorONLY_merged = tumorONLY_merged[['transcript_id','gene_id','gene_name','gene_type','Length','n']]
    tumorONLY_merged.sort_values(by=['n'], ascending=False).to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_n_")+proj+".csv",index=False)

BRCA
LUSC
PRAD
KIRC
KIRP
LUAD
         transcript_id   SRR5885319   SRR5885320   SRR5885321   SRR5885322  \
0       TCONS_00002128   918.580901   259.244976   743.867429   460.021127   
1      ENST00000387314  2739.699672   347.541269  1844.771818   399.913977   
2      ENST00000389680  4460.676619  1093.170912  3362.743728   897.971994   
3       TCONS_00002129  8547.016124  2386.216764  6447.811071  1860.799654   
4      ENST00000387342  1675.129075    55.297117  1637.638926   179.115198   
...                ...          ...          ...          ...          ...   
57861  ENST00000425340     0.290033     0.554147     0.284691     0.488158   
57862  ENST00000318083     0.768010     2.113511     0.516160     1.961413   
57863  ENST00000222145     4.884411    15.923205     4.564286    44.418950   
57864  ENST00000597553     0.075058     0.251131     0.010810     0.057150   
57865  ENST00000645652     1.206211     2.161318     0.857057     3.291924   

        SRR5885323   SRR5885324  

FileNotFoundError: [Errno 2] No such file or directory: '/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/cancers/featureCounts/table_of_counts_TPMs_GSE214846_LIHC.csv'

In [20]:
for proj in projects:
    cancer_type = proj[5:]
    print(cancer_type)

    fc = pd.read_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+cancer_type+".csv")
    tumor_transcripts = list()
    patients=pd.read_csv(os.path.join("/users/genomics/marta/TCGA_RNASeq",proj,"results/QC_patients.csv"))
    patients['normal'] = patients['normal'].str.replace('TCGA', cancer_type)
    patients['tumor'] = patients['tumor'].str.replace('TCGA', cancer_type)

    for index,patient in patients.iterrows():
        if "_" not in patient.iloc[0]: ## if there are two samples per patient, keep only one
            patient_fc = fc[["transcript_id","Length",patient.iloc[2]]]

            tumor_patient_fc = patient_fc[patient_fc[patient.iloc[2]] >= 1 ]
            tumor_transcripts.extend(tumor_patient_fc.transcript_id.values.tolist())

    tumor1FPKM = fc[fc['transcript_id'].isin(tumor_transcripts)]

    known = tumor1FPKM[tumor1FPKM['transcript_id'].str.contains('ENST')]
    novel = tumor1FPKM[tumor1FPKM['transcript_id'].str.contains('TCONS')]
    merged = tumor1FPKM.merge(transcript_gene, on=['transcript_id'], how="inner")
    merged = merged[merged['gene_type'].isin(['lncRNA','processed_pseudogene','novel','protein_coding'])]

    lncRNA = merged[merged['gene_type'].isin(['lncRNA','processed_pseudogene'])]
    print("lncRNA: ",len(lncRNA))

    cds = merged[merged['gene_type'] == "protein_coding"]
    print("PROTEIN CODING: ",len(cds))

    novel = merged[merged['gene_type'] == "novel"]
    novel = novel[novel['Length'].astype(int) < 91667]
    novel = novel[novel['Length'].astype(int) > 300]

    print("NOVEL: ",len(novel))

    # # tumor1FPKM = pd.concat([novel, lncRNA, cds], ignore_index = True)
    # merged.to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_table_of_counts_")+cancer_type+".csv",index=False)
    # merged[['transcript_id','gene_id','gene_name','gene_type','Length']].to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_")+cancer_type+".csv",index=False)

    tumorONLY_merged = merged[merged.columns.drop(list(merged.filter(regex='normal')))]
    ###### GET TUMOR-EXPRESSED > 1 TPM
    # Filter only integer columns
    int_columns = tumorONLY_merged.select_dtypes(include='int64')
    
    # Create a new column with the count of int columns per row whose value is > 1
    # tumorONLY_merged['n'] = int_columns.apply(lambda row: (row > 1).sum(), axis=1)
    tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)
    print(tumorONLY_merged)
    tumorONLY_merged = tumorONLY_merged[['transcript_id','gene_id','gene_name','gene_type','Length','n']]
    tumorONLY_merged.sort_values(by=['n'], ascending=False).to_csv(os.path.join(cancer_dir,"tumorexpressed/tumor_1FPKM_n_")+cancer_type+".csv",index=False)

BRCA
lncRNA:  10685
PROTEIN CODING:  16230
NOVEL:  478


/tmp/ipykernel_877837/2433886323.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)


         transcript_id  BRCA-A7-A0CE_tumor  BRCA-A7-A0CH_tumor  \
0       TCONS_00002128          267.709487          282.620818   
3       TCONS_00002129         1892.387570         3628.566131   
7      ENST00000361390         4922.762555         6200.533782   
11     ENST00000361453         4632.671860         5120.310994   
17     ENST00000361624        12520.123574        12805.411471   
...                ...                 ...                 ...   
33013  ENST00000600468            0.050594            0.058366   
33014  ENST00000425340            6.479699           10.709122   
33015  ENST00000318083            2.510292            1.574990   
33016  ENST00000222145            4.556231            4.455789   
33017  ENST00000645652            6.365758            1.211338   

       BRCA-A7-A0D9_tumor  BRCA-A7-A0DB_2_tumor  BRCA-A7-A0DB_3_tumor  \
0              361.264756            185.775806             11.623270   
3             4115.244777           3399.586815            96

/tmp/ipykernel_877837/2433886323.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)


         transcript_id  LUSC-22-4593_tumor  LUSC-22-4609_tumor  \
0       TCONS_00002128           51.003975          134.036441   
3       TCONS_00002129         1071.286059          908.034969   
7      ENST00000361390         2250.151348         2307.340567   
11     ENST00000361453         2767.517768         1845.464335   
17     ENST00000361624         8874.354968         5926.298466   
...                ...                 ...                 ...   
31495  ENST00000425340            2.286746           11.556526   
31496  ENST00000318083           17.205136            2.935649   
31497  ENST00000222145            4.531111           12.698763   
31498  ENST00000597553            0.055010            0.207989   
31499  ENST00000645652            7.776261           10.438391   

       LUSC-22-5471_tumor  LUSC-22-5472_tumor  LUSC-22-5478_tumor  \
0               37.784509          106.868616           61.738106   
3             1294.795172         1241.888811          838.418900   


...

         transcript_id  KIRC-A3-3358_tumor  KIRC-A3-3387_tumor  \
0       TCONS_00002128          304.946790          192.179407   
3       TCONS_00002129         4986.046589         1906.270954   
7      ENST00000361390        10920.358956         4938.040304   
11     ENST00000361453        10844.425826         4016.360207   
17     ENST00000361624        18364.112545        10536.273892   
...                ...                 ...                 ...   
27694  ENST00000594723            4.237029            2.839989   
27695  ENST00000084798            5.050094            8.333380   
27697  ENST00000318083            0.678024            1.242293   
27698  ENST00000222145            7.716036           13.531194   
27699  ENST00000645652            2.201132            2.337848   

       KIRC-B0-4700_tumor  KIRC-B0-4712_tumor  KIRC-B0-5402_tumor  \
0              149.322490          212.805185          165.340737   
3             5599.814533         5658.823521         5616.135322   


/tmp/ipykernel_877837/2433886323.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)


         transcript_id  KIRP-A4-A4ZT_tumor  KIRP-A4-A57E_tumor  \
0       TCONS_00002128          294.209363          161.241017   
3       TCONS_00002129         7291.239248        15292.071033   
7      ENST00000361390        11544.965302         4320.608463   
11     ENST00000361453         9045.966882         5539.460963   
17     ENST00000361624        28101.293761        36216.373746   
...                ...                 ...                 ...   
27451  ENST00000084798            8.700903            1.310541   
27453  ENST00000425340            0.249967            0.371580   
27454  ENST00000318083            0.463341            1.008063   
27455  ENST00000222145            2.114447           10.953976   
27456  ENST00000645652            0.263027            0.208092   

       KIRP-B9-4115_tumor  KIRP-BQ-5875_tumor  KIRP-BQ-5877_tumor  \
0              740.683401          521.762270         1138.027864   
3             3068.542507        17463.736697          563.632531   


/tmp/ipykernel_877837/2433886323.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumorONLY_merged['n'] = tumorONLY_merged.drop(columns=['transcript_id','gene_id','gene_name','gene_type','Length']).apply(count_greater_than_one, axis=1)


         transcript_id  LUAD-38-4625_tumor  LUAD-38-4626_tumor  \
0       TCONS_00002128           33.081218          167.607586   
3       TCONS_00002129          559.679343         1482.097508   
7      ENST00000361390         1205.134477         5597.900024   
11     ENST00000361453         2538.791386         6117.085762   
17     ENST00000361624         3116.938737        12994.530576   
...                ...                 ...                 ...   
31910  ENST00000084798           25.689259            7.925141   
31912  ENST00000425340            4.456336            5.032938   
31913  ENST00000318083            5.428071            1.052470   
31914  ENST00000222145            2.926995           13.684228   
31915  ENST00000645652            6.753087            5.483625   

       LUAD-38-4627_tumor  LUAD-38-4632_tumor  LUAD-44-2655_tumor  \
0              112.342463           46.514476          176.747297   
3             1980.756407         1469.437189         2027.146515   


In [21]:
tumorONLY_merged

,transcript_id,gene_id,gene_name,gene_type,Length,n
0,TCONS_00002128,XLOC_001837,XLOC_001837,novel,577,70
3,TCONS_00002129,XLOC_001838,XLOC_001838,novel,353,70
7,ENST00000361390,ENSG00000198888,MT-ND1,protein_coding,956,70
11,ENST00000361453,ENSG00000198763,MT-ND2,protein_coding,1042,70
17,ENST00000361624,ENSG00000198804,MT-CO1,protein_coding,1542,70
...,...,...,...,...,...,...
31910,ENST00000084798,ENSG00000063180,CA11,protein_coding,1715,65
31912,ENST00000425340,ENSG00000176920,FUT2,protein_coding,3116,69
31913,ENST00000318083,ENSG00000176909,MAMSTR,protein_coding,1858,40
31914,ENST00000222145,ENSG00000105538,RASIP1,protein_coding,3199,65


In [2]:
#Select normal samples and create a file with min max expression in normal samples per cancer type
tolerance_dictionary = dict()

for proj in projects:
    cancer_type = proj[5:]
    fc = pd.read_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+cancer_type+".csv")
    normals = [col for col in fc.columns if 'normal' in col]
    tolerance_dictionary[cancer_type] = len(normals)*0.05

In [3]:
tolerance_dictionary

{'BRCA': 5.6000000000000005,
 'LUSC': 2.45,
 'PRAD': 2.6,
 'KIRC': 3.5500000000000003,
 'KIRP': 1.6,
 'LUAD': 2.9000000000000004}

In [14]:
## TUMOR-SPECIFIC
for proj in projects:
    tumorspecific_candidates = pd.DataFrame()
    cancer_type = proj[5:]
    print(cancer_type)

    ## import table of counts
    fc = pd.read_csv(os.path.join(cancer_dir,"featureCounts/table_of_counts_TPMs_")+cancer_type+".csv")
    fc_info = fc.merge(transcript_gene, on=['transcript_id'], how="inner")
    fc_info = fc_info[fc_info['gene_type'].isin(['lncRNA','processed_pseudogene','novel','protein_coding'])]

    ## table of counts normal samples
    normals = [col for col in fc_info.columns if 'normal' in col]
    fc_normals = fc_info[normals]
    # Function to count values above 0.1 in a row
    def count_values_above_01(row):
        return sum(row > 0.1)
    fc_normals['counts_01'] = fc_normals.apply(count_values_above_01, axis=1)
    fc_normals_info = pd.concat([fc_normals, fc_info[['gene_id','transcript_id','gene_name','gene_type']]], axis=1)
    fc_normals_info = fc_normals_info[['gene_id','transcript_id','gene_name','gene_type','counts_01']]

    ## import patients data
    patients=pd.read_csv(os.path.join("/users/genomics/marta/TCGA_RNASeq",proj,"results/QC_patients.csv"))
    patients['normal'] = patients['normal'].str.replace('TCGA', cancer_type)
    patients['tumor'] = patients['tumor'].str.replace('TCGA', cancer_type)

    for index,patient in patients.iterrows():
        if "_" not in patient.iloc[0]: ## if there are two samples per patient, keep only one
            patient_fc = fc_info[["transcript_id","gene_id","gene_name","gene_type","Length",patient.iloc[1],patient.iloc[2]]] ## patient - normal - tumor
            tumor1TPM = patient_fc[patient_fc[patient.iloc[2]] > 1 ]
            tumor1TPM_normal01TPM = tumor1TPM[tumor1TPM[patient.iloc[1]] < 0.1 ]
            try:
                os.makedirs(os.path.join(cancer_dir,"tumorspecific",cancer_type,patient.iloc[0]))
            except:
                pass
            tumor1TPM_normal01TPM.to_csv(os.path.join(cancer_dir,"tumorspecific",cancer_type,patient.iloc[0],"tumorspecific_TPMs.csv"), index=None)
            tumorspecific_candidates = pd.concat([tumorspecific_candidates,tumor1TPM_normal01TPM[["transcript_id","gene_id","gene_name","gene_type","Length"]]])
summary = tumorspecific_candidates.groupby(["transcript_id","gene_id","gene_name","gene_type","Length"]).size().reset_index(name='n')
summary.sort_values(by="n", ascending=False).to_csv(os.path.join(cancer_dir,"tumorspecific")+"/tumorspecific_n_"+cancer_type+".csv", index=False)
print(len(summary))
### tolerance 5% only
summary_tolerance = fc_normals_info.merge(summary, on=["gene_id","transcript_id"], how="right")
# ## Expression > 0.1 in less than 5%
summary_tolerance_accepted = summary_tolerance[summary_tolerance['counts_01'] < tolerance_dictionary[cancer_type]]
summary_tolerance_accepted.drop(["counts_01"], axis=1, inplace=True)
summary_tolerance_accepted.sort_values(by="n", ascending=False).to_csv(os.path.join(cancer_dir,"tumorspecific")+"/tumorspecific_n_"+cancer_type+"_TOLERANCE5PERCENT.csv", index=False)
print(len(summary_tolerance_accepted))
print("Done!")

BRCA


/tmp/ipykernel_877837/4218154473.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_normals['counts_01'] = fc_normals.apply(count_values_above_01, axis=1)


LUSC


/tmp/ipykernel_877837/4218154473.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_normals['counts_01'] = fc_normals.apply(count_values_above_01, axis=1)


PRAD


...

LUAD


/tmp/ipykernel_877837/4218154473.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_normals['counts_01'] = fc_normals.apply(count_values_above_01, axis=1)


5956


/tmp/ipykernel_877837/4218154473.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_tolerance_accepted.drop(["counts_01"], axis=1, inplace=True)


1225
Done!
